In [3]:
## Import the  pyspark libraries adn create the pyspark session
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Rainfall-predictions-ML-project').getOrCreate()
spark
